In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.WEEKLY_STOCK_SPECULATION)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

## Historical Training and Prediction Set

In [6]:
# market.connect()
# training_sets = []
# for ticker in tqdm(sp500["ticker"].unique()):
#     try:
#         prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
#         prices = p.column_date_processing(prices)
#         ticker_data = pricer_class.training_set(ticker,prices,False)
#         training_sets.append(ticker_data)
#     except Exception as e:
#         print(str(e))
#         continue
# market.disconnect()
# data = pd.concat(training_sets)
# training_data = data.dropna().copy().sort_values(["year","week"])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:35<00:00, 13.94it/s]


## Historical Modeling and Simulation Code

In [8]:
## historical model and simulation
# pricer_class.db.connect()
# model_start_date = backtest_start_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
# model_end_date = backtest_end_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
# for year in range(model_start_date.year,model_end_date.year):
#     training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
#     prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
#     stuff = modeler_strat.model(training_slice,prediction_set,pricer_class.factors,False)
#     stuff = stuff.rename(columns={"prediction":f"price_prediction"})
#     relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
#     stuff = pricer_class.sim_processor(stuff)
#     pricer_class.db.store("sim",stuff[relevant_columns])
# pricer_class.db.disconnect()

## Current Year Modeling Code

In [9]:
# pricer_class.db.connect()
# training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
# prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
# stuff = modeler_strat.recommend_model(training_slice,pricer_class.factors,False)
# stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]
# stuff["year"] = current_year
# pricer_class.db.store("models",stuff)
# pricer_class.db.disconnect()

In [10]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices,True)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
data = pd.concat(training_sets)
training_data = data.dropna().copy().sort_values(["year","week"])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:35<00:00, 13.91it/s]


In [11]:
training_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,year,week,ticker,adjclose
13,25.077294,24.083908,24.008833,22.553571,22.576964,23.736821,24.387693,23.652944,23.349112,24.923416,25.352133,26.085898,27.231891,26.158747,2000,14,MMM,25.077294
13,2.011732,1.930327,1.888632,1.738618,1.778548,1.858409,1.942185,1.901427,1.986582,2.131963,2.311003,2.399871,2.383116,2.356966,2000,14,AOS,2.011732
13,9.418513,8.651546,8.481721,8.125039,7.718261,8.309892,8.097987,8.604456,8.673087,8.325422,7.615064,8.143700,8.624169,8.669194,2000,14,ABT,9.418513
13,19.338000,21.370000,24.750000,25.257000,31.413000,34.757000,36.436250,35.838000,28.851000,28.088000,29.488000,25.828750,20.457000,18.020000,2000,14,ABMD,19.338000
13,0.761955,0.951261,0.987468,0.983182,1.035496,1.020570,0.966113,1.033427,1.087515,1.229384,1.166577,1.194988,1.265442,1.185050,2000,14,ATVI,0.761955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,133.226000,130.815000,130.980778,138.102000,137.546000,139.250000,139.020000,137.526000,132.838000,132.212500,129.322000,128.116000,126.726000,126.726000,2023,23,YUM,133.226000
1222,273.892000,268.640000,267.430000,273.776000,270.246000,268.038000,284.972000,294.454000,303.330000,305.192500,300.448000,290.172000,289.752000,300.028000,2023,23,ZBRA,273.892000
1142,133.022000,127.735000,129.932000,135.202000,136.870000,139.200000,138.614000,136.434000,131.202000,128.430000,126.920144,125.862528,124.938286,122.878206,2023,23,ZBH,133.022000
1222,30.641000,28.235000,28.686000,25.505641,23.508000,23.418000,27.166000,30.916000,30.274000,28.712500,30.110000,29.658000,30.808000,44.718000,2023,23,ZION,30.641000


In [12]:
pricer_class.db.connect()
pricer_class.db.drop("predictions")
prediction_set = training_data[training_data["year"]==current_year].reset_index(drop=True)
models = pricer_class.db.retrieve("models")
stuff = modeler_strat.recommend(models,prediction_set,pricer_class.factors)
stuff = stuff.rename(columns={"prediction":f"price_prediction"})
stuff = pricer_class.sim_processor(stuff)
relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
pricer_class.db.store("predictions",stuff[relevant_columns])
pricer_class.db.disconnect()

In [13]:
stuff

,0,1,2,3,4,5,6,7,8,9,...,13,year,week,ticker,adjclose,xgb_prediction,xgb_score,cat_prediction,cat_score,price_prediction
0,122.681938,118.167401,120.018608,123.209868,123.805885,124.279542,126.621674,126.381644,124.063142,121.263875,...,109.756642,2023,2,MMM,122.681938,122.361809,0.998962,122.743995,0.940958,122.552902
1,59.538274,56.799117,56.025417,57.380631,58.484219,59.805750,61.096076,59.653190,56.905117,54.174876,...,50.200813,2023,2,AOS,59.538274,58.840855,0.998962,58.560693,0.940958,58.700774
2,109.932934,108.095435,106.471899,108.508501,104.704730,105.790390,104.437773,102.065865,100.427470,97.235868,...,100.619729,2023,2,ABT,109.932934,110.617622,0.998962,106.709728,0.940958,108.663675
3,160.983578,159.334755,159.121291,160.671970,161.370758,157.639313,156.027293,149.923211,146.064178,142.805789,...,136.582506,2023,2,ABBV,160.983578,160.995438,0.998962,159.412588,0.940958,160.204013
4,381.020000,381.032500,380.316000,379.092000,377.398000,377.620000,375.370000,373.612000,350.206000,263.692000,...,246.312000,2023,2,ABMD,381.020000,386.094177,0.998962,377.200678,0.940958,381.647428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11216,133.226000,130.815000,130.980778,138.102000,137.546000,139.250000,139.020000,137.526000,132.838000,132.212500,...,126.726000,2023,24,YUM,133.226000,132.264206,0.998962,132.158847,0.940958,132.211526
11217,273.892000,268.640000,267.430000,273.776000,270.246000,268.038000,284.972000,294.454000,303.330000,305.192500,...,300.028000,2023,24,ZBRA,273.892000,276.956360,0.998962,267.463693,0.940958,272.210026
11218,133.022000,127.735000,129.932000,135.202000,136.870000,139.200000,138.614000,136.434000,131.202000,128.430000,...,122.878206,2023,24,ZBH,133.022000,132.264206,0.998962,132.158847,0.940958,132.211526
11219,30.641000,28.235000,28.686000,25.505641,23.508000,23.418000,27.166000,30.916000,30.274000,28.712500,...,44.718000,2023,24,ZION,30.641000,29.935337,0.998962,31.995942,0.940958,30.965639
